In [1]:
import pandas as pd

# 读取数据
mig = pd.read_csv('../data/processed/migration_flow_cleaned.csv')
spei = pd.read_csv('../data/processed/spei03_country_month_cleaned.csv')

In [2]:
# 检查主键字段类型
print("迁移数据主键字段类型：\n", mig.dtypes)
print("\nSPEI数据主键字段类型：\n", spei.dtypes)


迁移数据主键字段类型：
 origin_iso3          object
destination_iso3     object
migration_month      object
year                  int64
month                 int64
flow                  int64
log_flow            float64
origin_iso2          object
destination_iso2     object
dtype: object

SPEI数据主键字段类型：
 country     object
ISO_A3      object
date        object
spei       float64
dtype: object


In [3]:
# 检查国家代码格式
assert all(mig['origin_iso3'].str.len() == 3)
assert all(spei['ISO_A3'].str.len() == 3)

# 找出在mig中但不在spei中的国家代码
missing_codes = set(mig['origin_iso3']) - set(spei['ISO_A3'])
print("以下国家在SPEI数据中缺失：", missing_codes)

以下国家在SPEI数据中缺失： {'SGP', 'MDV', 'MLT', 'STP', 'KIR', 'TON', 'LCA', 'MAC', 'HKG', 'GRD', 'AND', 'BRB', 'FSM', 'BHR'}


In [4]:
# 1. 读取或获取missing_countries_gdf
missing_countries_gdf = pd.read_csv('../data/processed/spei_missing_countries.csv')  # 或直接用变量

# 2. 检查missing_codes哪些在missing_countries_gdf里
missing_codes = {'GRD', 'STP', 'LCA', 'SGP', 'MAC', 'TON', 'MLT', 'MDV', 'BRB', 'HKG', 'AND', 'BHR', 'FSM', 'KIR'}

# 属于spei_missing_countries的
in_missing = [code for code in missing_codes if code in set(missing_countries_gdf['ISO_A3'])]
print("属于spei_missing_countries的：", in_missing)

# 不属于spei_missing_countries的
not_in_missing = [code for code in missing_codes if code not in set(missing_countries_gdf['ISO_A3'])]
print("不属于spei_missing_countries的：", not_in_missing)

属于spei_missing_countries的： ['SGP', 'MDV', 'STP', 'MLT', 'TON', 'LCA', 'MAC', 'HKG', 'GRD', 'AND', 'BRB', 'FSM', 'BHR']
不属于spei_missing_countries的： ['KIR']


检查了一下，除了KIR，其他都是因为国家太小，没有spei网格点数。

In [5]:
# 检查缺失值
print(mig.isnull().sum())
print(spei.isnull().sum())

origin_iso3         0
destination_iso3    0
migration_month     0
year                0
month               0
flow                0
log_flow            0
origin_iso2         0
destination_iso2    0
dtype: int64
country    0
ISO_A3     0
date       0
spei       0
dtype: int64


In [6]:
# 检查重复
print(mig.duplicated(subset=['origin_iso3', 'destination_iso3', 'year', 'month']).sum())
print(spei.duplicated(subset=['ISO_A3', 'date']).sum())

0
0


In [7]:
# 检查主键唯一性
assert mig.duplicated(subset=['origin_iso3', 'destination_iso3', 'year', 'month']).sum() == 0
assert spei.duplicated(subset=['ISO_A3', 'date']).sum() == 0

In [8]:
# 处理 SPEI 字段
spei['date'] = pd.to_datetime(spei['date'])
spei['year'] = spei['date'].dt.year
spei['month'] = spei['date'].dt.month
spei = spei.rename(columns={'ISO_A3': 'origin_iso3','spei':'origin_spei'})
spei = spei[['origin_iso3', 'year', 'month', 'origin_spei']]

# 处理mig字段
mig = mig.rename(columns={'migration_month':'migration_date'})
mig = mig[['origin_iso3', 'destination_iso3', 'year', 'month', 'migration_date','flow','log_flow']]

# 合并
mig_spei = mig.merge(spei, on=['origin_iso3', 'year', 'month'], how='left')

# 检查合并效果
print(mig_spei.head())
print("合并后缺失值统计：\n", mig_spei.isnull().sum())

  origin_iso3 destination_iso3  year  month migration_date  flow  log_flow  \
0         AND              ARE  2019      1     2019-01-01    12  2.564949   
1         AND              ARE  2019      2     2019-02-01     2  1.098612   
2         AND              ARE  2019      3     2019-03-01     1  0.693147   
3         AND              ARE  2019      4     2019-04-01     7  2.079442   
4         AND              ARE  2019      5     2019-05-01     0  0.000000   

   origin_spei  
0          NaN  
1          NaN  
2          NaN  
3          NaN  
4          NaN  
合并后缺失值统计：
 origin_iso3              0
destination_iso3         0
year                     0
month                    0
migration_date           0
flow                     0
log_flow                 0
origin_spei         120288
dtype: int64


In [9]:
# 保存
mig_spei.to_csv('../data/processed/migration_spei.csv', index=False)
print("已保存 migration_spei.csv")

已保存 migration_spei.csv


现在合并MA(origin)

In [10]:
# 检查 MA 数据主键字段类型
ma = pd.read_csv('../data/processed/market_access_panel.csv')
print("MA 数据主键字段类型：\n", ma.dtypes)

MA 数据主键字段类型：
 iso3     object
year      int64
MA      float64
dtype: object


In [11]:
# 检查 MA 数据缺失值
print("MA 数据缺失值统计：\n", ma.isnull().sum())

MA 数据缺失值统计：
 iso3    0
year    0
MA      0
dtype: int64


In [12]:
# 1. 统计 MA 数据
ma_countries = ma['iso3'].nunique()  # MA 数据中的国家数量
ma_rows = len(ma)  # MA 数据的总行数
print(f"MA 数据包含 {ma_countries} 个国家，共 {ma_rows} 条数据。")

# 2. 统计 migration_spei 数据
mig_spei_countries = mig_spei['origin_iso3'].nunique()  # migration_spei 中的国家数量
mig_spei_rows = len(mig_spei)  # migration_spei 的总行数
print(f"migration_spei 数据包含 {mig_spei_countries} 个国家，共 {mig_spei_rows} 条数据。")

# 3. 统计 mig 数据
mig_countries = mig['origin_iso3'].nunique()  # mig 数据中的国家数量
mig_rows = len(mig)  # mig 数据的总行数
print(f"mig 数据包含 {mig_countries} 个国家，共 {mig_rows} 条数据。")

MA 数据包含 199 个国家，共 796 条数据。
migration_spei 数据包含 180 个国家，共 1545874 条数据。
mig 数据包含 180 个国家，共 1545874 条数据。


In [13]:
# 检查 MA 数据与迁移数据的国家代码一致性
ma_countries = set(ma['iso3'])
mig_countries = set(mig['origin_iso3'])
missing_in_ma = mig_countries - ma_countries
print("以下国家在 MA 数据中缺失：", missing_in_ma)

以下国家在 MA 数据中缺失： {'XKX', 'TLS', 'SSD', 'TWN', 'ROU', 'COD', 'MNE', 'SRB'}


In [14]:
# 获取 migration_spei 和 MA 数据中的国家代码集合
mig_spei_countries = set(mig_spei['origin_iso3'])
ma_countries = set(ma['iso3'])

# 找出独有的国家
only_in_mig_spei = mig_spei_countries - ma_countries
only_in_ma = ma_countries - mig_spei_countries

# 打印数量
print(f"仅在 migration_spei 中的国家数量：{len(only_in_mig_spei)}")
print(f"仅在 MA 数据中的国家数量：{len(only_in_ma)}")

# 打印名字
print("仅在 migration_spei 中的国家：", only_in_mig_spei)
print("仅在 MA 数据中的国家：", only_in_ma)

仅在 migration_spei 中的国家数量：8
仅在 MA 数据中的国家数量：27
仅在 migration_spei 中的国家： {'XKX', 'TLS', 'SSD', 'TWN', 'ROU', 'COD', 'MNE', 'SRB'}
仅在 MA 数据中的国家： {'GIB', 'SMR', 'MHL', 'NCL', 'CHN', 'CUB', 'ABW', 'BMU', 'SYC', 'VGB', 'DMA', 'PRI', 'SOM', 'IRN', 'GRL', 'TCA', 'PLW', 'CYM', 'TUV', 'FRO', 'NRU', 'PRK', 'NAM', 'ATG', 'PYF', 'KNA', 'MNP'}


对比了之后发现，这是因为计算MA需要的dist_cepii文件缺少这些国家，合并之后直接剔除比较好

In [ ]:
# 重命名 MA 数据的列，确保字段一致
ma = ma.rename(columns={'iso3': 'origin_iso3', 'MA': 'origin_ma'})

# 扩展 MA 数据到每月
ma = ma.loc[ma.index.repeat(12)]  # 每年重复 12 次
ma['month'] = list(range(1, 13)) * (len(ma) // 12)  # 添加 month 列

# 合并 MA 数据到 migration_spei
mig_spei_ma = mig_spei.merge(ma, on=['origin_iso3', 'year', 'month'], how='left')

# 检查合并效果
print(mig_spei_ma.head())
print("合并后缺失值统计：\n", mig_spei_ma.isnull().sum())



  origin_iso3 destination_iso3  year  month migration_date  flow  log_flow  \
0         AND              ARE  2019      1     2019-01-01    12  2.564949   
1         AND              ARE  2019      2     2019-02-01     2  1.098612   
2         AND              ARE  2019      3     2019-03-01     1  0.693147   
3         AND              ARE  2019      4     2019-04-01     7  2.079442   
4         AND              ARE  2019      5     2019-05-01     0  0.000000   

   origin_spei     origin_ma  
0          NaN  4.296489e+08  
1          NaN  4.296489e+08  
2          NaN  4.296489e+08  
3          NaN  4.296489e+08  
4          NaN  4.296489e+08  
合并后缺失值统计：
 origin_iso3              0
destination_iso3         0
year                     0
month                    0
migration_date           0
flow                     0
log_flow                 0
origin_spei         120288
origin_ma            68600
dtype: int64
已保存 migration_spei_ma.csv


In [ ]:
# 保存合并后的数据
mig_spei_ma.to_csv('../data/processed/migration_spei_ma.csv', index=False)
print("已保存 migration_spei_ma.csv")
# 保存为压缩文件
mig_spei_ma.to_csv('../data/processed/migration_spei_ma_optimized.csv.gz', index=False, compression='gzip')
print("已保存优化后的压缩文件：migration_spei_ma_optimized.csv.gz")

已保存优化后的压缩文件：migration_spei_ma_optimized.csv.gz


ok现在合并边境摩擦

In [16]:
# 读取 Border Friction 数据
bf = pd.read_csv('../data/processed/border_friction_panel_ew.csv')

# 检查主键字段类型
print("Border Friction 数据主键字段类型：\n", bf.dtypes)

# 检查缺失值
print("Border Friction 数据缺失值统计：\n", bf.isnull().sum())

Border Friction 数据主键字段类型：
 iso_o               object
iso_d               object
contig               int64
comlang_off          int64
colony               int64
border_friction    float64
dtype: object
Border Friction 数据缺失值统计：
 iso_o              0
iso_d              0
contig             0
comlang_off        0
colony             0
border_friction    0
dtype: int64


In [17]:
# 检查列的最大值、最小值和小数位数
print(mig_spei_ma[['flow', 'log_flow', 'origin_ma']].describe())

               flow      log_flow     origin_ma
count  1.545874e+06  1.545874e+06  1.477274e+06
mean   7.678771e+01  1.618147e+00  1.683872e+08
std    9.029347e+02  1.754999e+00  1.731478e+08
min    0.000000e+00  0.000000e+00  0.000000e+00
25%    0.000000e+00  0.000000e+00  6.743269e+07
50%    3.000000e+00  1.386294e+00  1.096172e+08
75%    1.200000e+01  2.564949e+00  1.934012e+08
max    1.639370e+05  1.200724e+01  1.131320e+09
